In [1]:
import torch

### Are GPU's available?

In [2]:
gpus = False
n_gpus = 0

if torch.cuda.is_available():
    gpus = True
    n_gpus = torch.cuda.device_count()

In [4]:
n_gpus

1

In [5]:
!rm tiny-imagenet-200/*.txt

rm: cannot remove ‘tiny-imagenet-200/*.txt’: No such file or directory


In [6]:
!rm tiny-imagenet-200/val/*.txt

rm: cannot remove ‘tiny-imagenet-200/val/*.txt’: No such file or directory


In [7]:
from torchvision import transforms

In [8]:
from torchvision import transforms, datasets

data_transform = transforms.Compose([
        transforms.ToTensor()
    ])

train_dataset = datasets.ImageFolder(root='tiny-imagenet-200/train',
                                           transform=data_transform)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 128,
                                             shuffle=True
                                          )

In [9]:
val_dataset = datasets.ImageFolder(root='tiny-imagenet-200/val',
                                           transform=data_transform)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                             shuffle=False
                                          )

In [10]:
test_dataset = datasets.ImageFolder(root='tiny-imagenet-200/test',
                                           transform=data_transform)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                             shuffle=False
                                          )

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Uncomment for parallelism
        self.resnet_50 = nn.DataParallel(models.resnet50(pretrained = True),device_ids = [i for i in range(n_gpus)])
        #self.resnet_50 = models.resnet50(pretrained = True)
        self.output = nn.Linear(1000, 200)
        

    def forward(self, x):
        x = self.resnet_50(x)
        x = F.relu(x)
        # No need for softmax as CrossEntropy already implements it
        x = self.output(x)
        return x

net = Net()
net.train(True)

Net(
  (resnet_50): DataParallel(
    (module): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [13]:
if gpus:
    net = net.to('cuda:0')

In [14]:
import torch.optim as optim
from torch import nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
from time import time

start = time()

for epoch in range(2):
    start_epoch = time()
    running_loss=0.0
    for i, data in enumerate(train_loader, start = 0):
        inputs, labels = data
        if gpus:
            labels = labels.to('cuda:0') #Move to GPU
        start_mini = time()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        end_mini = time()
        
        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 1000 mini-batches
            print('epoch %d, minibatch number %5d training loss: %.3f and time taken is %.3f seconds' %
                  (epoch + 1, i + 1, running_loss / 10,(end_mini - start_mini)))
            running_loss = 0.0
            
    end_epoch = time()
    print("Epoch %d took %.3f minutes"%(epoch + 1,(end_epoch - start_epoch )/60))
    print("")

end = time()    
print("")
print('Finished Training after %.3f hours'%((end-start) / 60 / 60 ))

epoch 1, minibatch number    10 training loss: 6.147 and time taken is 0.112 seconds
epoch 1, minibatch number    20 training loss: 5.742 and time taken is 0.089 seconds
epoch 1, minibatch number    30 training loss: 5.386 and time taken is 0.133 seconds
epoch 1, minibatch number    40 training loss: 5.185 and time taken is 0.108 seconds
epoch 1, minibatch number    50 training loss: 4.962 and time taken is 0.096 seconds
epoch 1, minibatch number    60 training loss: 4.669 and time taken is 0.091 seconds
epoch 1, minibatch number    70 training loss: 4.393 and time taken is 0.090 seconds
epoch 1, minibatch number    80 training loss: 4.186 and time taken is 0.090 seconds
epoch 1, minibatch number    90 training loss: 4.009 and time taken is 0.106 seconds
epoch 1, minibatch number   100 training loss: 3.702 and time taken is 0.091 seconds


KeyboardInterrupt: 

In [ ]:
def fetch_accuracy(loader):
    with torch.no_grad():
            net.to('cuda:0')
            net.eval()
            count = 0
            for inputs in loader:
                inputs ,labels = inputs
                if gpus:
                    inputs = inputs.to('cuda:0')
                    labels = labels.to('cuda:0')
                predictions = net(inputs)
                _, predicted = torch.max(predictions, 1)
                if predicted == labels:
                    count = count + 1
            return count / len(loader)

In [ ]:
print("Test accuracy:")
print(fetch_accuracy(test_loader))